In [1]:
import os
import sys

PROJECT_ROOT = os.path.abspath(os.path.join(
                  os.path.dirname("test-pretrained"), 
                  os.pardir)
)
sys.path.append(PROJECT_ROOT)

import pandas as pd
import numpy as np
import torch
import json
import random
import torch.nn as nn
import torch.nn.functional as F
from copy import deepcopy
from src.data_augmentation import Mixup, Specmix, Cutmix
from src.features import extract_wavelet_from_raw_audio
from src.dataset import create_dataloader
from src.utils import feature_extraction_pipeline, read_features_files, choose_model, read_feature, pad_features
from src.models.utils import SaveBestModel, weight_init
from src.models.cnn3 import *
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from typing import Dict, Tuple, List, Union, Iterable
from sklearn.metrics import classification_report

In [2]:
def train(
    model: nn.Module,
    dataloader: DataLoader,
    optimizer: torch.optim.Adam,
    loss: torch.nn.CrossEntropyLoss,
    device: torch.device,
    mixer: Union[None, Mixup, Specmix, Cutmix]
) -> Tuple[float, float]:
    """
    Function responsible for the model training.

    Args:
        model (nn.Module): the created model.
        dataloader (DataLoader): the training dataloader.
        optimizer (torch.optim.Adam): the optimizer used.
        loss (torch.nn.CrossEntropyLoss): the loss function used.
        device (torch.device): which device to use.

    Returns:
        Tuple[float, float]: the training f1 and loss, respectively.
    """
    model.train()
    predictions = []
    targets = []
    train_loss = 0.0
    
    for index, (batch) in enumerate(dataloader, start=1):
        data = batch["features"].to(device)
        target = batch["labels"].to(device)
        optimizer.zero_grad()
        
        data = data.to(dtype=torch.float32)
        target = target.to(dtype=torch.float32)
        
        if not mixer is None:
            data, target = mixer(
                x=data,
                y=target
            )
            
        output = model(data)

        l = loss(output, target)
        train_loss += l.item()
        
        l.backward()
        optimizer.step()
        
        prediction = output.argmax(dim=-1, keepdim=True).to(dtype=torch.int)
        prediction = prediction.detach().cpu().numpy()
        predictions.extend(prediction.tolist())
        
        target = target.argmax(dim=-1, keepdim=True).to(dtype=torch.int)
        target = target.detach().cpu().numpy()
        targets.extend(target.tolist())
        
    train_loss = train_loss/index
    train_f1 = classification_report(
        targets,
        predictions,
        digits=6,
        output_dict=True,
        zero_division=0.0
    )
    train_f1 = train_f1["macro avg"]["f1-score"]
    return train_f1, train_loss

def evaluate(
    model: nn.Module,
    dataloader: DataLoader,
    loss: torch.nn.CrossEntropyLoss,
    device: torch.device
) -> Tuple[float, float]:
    """
    Function responsible for the model evaluation.

    Args:
        model (nn.Module): the created model.
        dataloader (DataLoader): the validaiton dataloader.
        loss (torch.nn.CrossEntropyLoss): the loss function used.
        device (torch.device): which device to use.

    Returns:
        Tuple[float, float]: the validation f1 and loss, respectively.
    """
    model.eval()
    predictions = []
    targets = []
    validation_loss = 0.0
    validation_f1 = []
    
    with torch.inference_mode():
        for index, (batch) in enumerate(dataloader):
            data = batch["features"].to(device)
            target = batch["labels"].to(device)

            data = data.to(dtype=torch.float32)
            target = target.to(dtype=torch.float32)
                        
            output = model(data)
            
            l = loss(output, target)
            validation_loss += l.item()
            
            prediction = output.argmax(dim=-1, keepdim=True).to(dtype=torch.int)
            prediction = prediction.detach().cpu().numpy()
            predictions.extend(prediction.tolist())
            
            target = target.argmax(dim=-1, keepdim=True).to(dtype=torch.int)
            target = target.detach().cpu().numpy()
            targets.extend(target.tolist())
    
    validation_loss = validation_loss/index
    validation_f1 = classification_report(
        targets,
        predictions,
        digits=6,
        output_dict=True,
        zero_division=0.0
    )
    validation_f1 = validation_f1["macro avg"]["f1-score"]
    return validation_f1, validation_loss

def test(
    model: nn.Module,
    dataloader: DataLoader,
    device: torch.device
) -> str:
    """
    Function responsible for the model testing in the test dataset.

    Args:
        model (nn.Module): the created model.
        dataloader (DataLoader): the test dataloader.
        device (torch.device): which device to use.

    Returns:
        str: the test classification report.
    """
    model.eval()
    predictions = []
    targets = []
    
    with torch.inference_mode():
        for batch in dataloader:
            data = batch["features"].to(device)
            target = batch["labels"].to(device)

            data = data.to(dtype=torch.float32)
            target = target.to(dtype=torch.float32)
            
            output = model(data)
            
            prediction = output.argmax(dim=-1, keepdim=True).to(dtype=torch.int)
            prediction = prediction.detach().cpu().numpy()
            predictions.extend(prediction.tolist())
            
            target = target.argmax(dim=-1, keepdim=True).to(dtype=torch.int)
            target = target.detach().cpu().numpy()
            targets.extend(target.tolist())
    
    class_report = classification_report(
        targets,
        predictions,
        digits=4,
        output_dict=True
    )
    return class_report

In [3]:
features_path = "../features8k/propor2022/"

# loading training features
X_train = read_feature(path=features_path, fold="0", name="X_train.pth")
y_train = read_feature(path=features_path, fold="0", name="y_train.pth")
print(f"Train: {X_train.shape}, {y_train.shape}")

# loading validation features
X_valid = read_feature(path=features_path, fold="0", name="X_valid.pth")
y_valid = read_feature(path=features_path, fold="0", name="y_valid.pth")
print(f"Valid: {X_valid.shape}, {y_valid.shape}")

# loading testing features
X_test = read_feature(path=features_path, fold=None, name="X_test.pth")
y_test = read_feature(path=features_path, fold=None, name="y_test.pth")
print(f"Test: {X_test.shape}, {y_test.shape}")

Train: torch.Size([500, 1, 128000]), torch.Size([500, 3])
Valid: torch.Size([125, 1, 128000]), torch.Size([125, 3])
Test: torch.Size([308, 1, 128000]), torch.Size([308, 3])


In [4]:
# reading the parameters configuration file
params = json.load(open("../config/mode_1.json", "r"))

feature_config = params["feature"]
feature_config["sample_rate"] = int(params["sample_rate"])
data_augmentation_config = params["data_augmentation"]
dataset = params["dataset"]
wavelet_config = params["wavelet"]
mode = params["mode"]

model_config = params["model"]
model_config["name"] = "cnn3"
model_config["use_gpu"] = False
model_config["learning_rate"] = 0.0001
device = torch.device("cuda" if torch.cuda.is_available and model_config["use_gpu"] else "cpu")

if dataset == "propor2022":
    if data_augmentation_config["target"] == "majority":
        data_augment_target = [0]
    elif data_augmentation_config["target"] == "minority":
        data_augment_target = [1, 2]
    elif data_augmentation_config["target"] == "all":
        data_augment_target = [0, 1, 2]
    else:
        raise ValueError("Invalid arguments for target. Should be 'all', 'majority' or 'minority")
else:
    raise NotImplementedError
    
model = choose_model(
    mode=mode,
    model_name=model_config["name"],
    device=device,
    dataset=dataset
)

/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


In [5]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        #return self.factor * \ (self.model_size ** (-0.5) * min(step ** (-0.5), step * self.warmup ** (-1.5)))
        return 0.0001
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [6]:
optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=model_config["learning_rate"],
    betas=(0.9, 0.98),
    eps=1e-9
)
noamopt = NoamOpt(512, 1, 400, optimizer)
loss = torch.nn.CrossEntropyLoss()
scheduler = None
mixer = None

# creating the model checkpoint object
sbm = SaveBestModel(
    output_dir=os.path.join(model_config["output_path"], dataset, mode, model_config["name"]),
    model_name=model_config["name"]
)

if model_config["use_lr_scheduler"]:
    print("\nWARNING: Using learning rate scheduler!\n")
    scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

if "mixup" in data_augmentation_config["techniques"].keys():
    print("\nWARNING: Using mixup data augmentation technique!\n")
    mixer = Mixup(
        alpha=data_augmentation_config["techniques"]["mixup"]["alpha"]
    )

if "specmix" in data_augmentation_config["techniques"].keys():
    print("\nWARNING: Using specmix data augmentation technique!\n")
    mixer = Specmix(
        p=data_augmentation_config["p"],
        min_band_size=data_augmentation_config["techniques"]["specmix"]["min_band_size"],
        max_band_size=data_augmentation_config["techniques"]["specmix"]["max_band_size"],
        max_frequency_bands=data_augmentation_config["techniques"]["specmix"]["max_frequency_bands"],
        max_time_bands=data_augmentation_config["techniques"]["specmix"]["max_time_bands"],
        device=device
    )

if "cutmix" in data_augmentation_config["techniques"].keys():
    print("\nWARNING: Using cutmix data augmentation technique!\n")
    mixer = Cutmix(
        alpha=data_augmentation_config["techniques"]["cutmix"]["alpha"],
        p=data_augmentation_config["p"]
    )

# creating the training dataloader
training_dataloader = create_dataloader(
    X=X_train,
    y=y_train,
    feature_config=feature_config,
    wavelet_config=wavelet_config,
    data_augmentation_config=data_augmentation_config,
    num_workers=0,
    mode=mode,
    shuffle=False,
    training=True,
    batch_size=model_config["batch_size"],
    data_augment_target=data_augment_target
)

# creating the validation dataloader
validation_dataloader = create_dataloader(
    X=X_valid,
    y=y_valid,
    feature_config=feature_config,
    wavelet_config=wavelet_config,
    data_augmentation_config=data_augmentation_config,
    num_workers=0,
    mode=mode,
    shuffle=False,
    training=False,
    batch_size=model_config["batch_size"],
    data_augment_target=data_augment_target
)

# creating the test dataloader
test_dataloader = create_dataloader(
    X=X_test,
    y=y_test,
    feature_config=feature_config,
    wavelet_config=wavelet_config,
    data_augmentation_config=None,
    num_workers=0,
    mode=params["mode"],
    shuffle=False,
    training=False,
    batch_size=params["model"]["batch_size"],
    data_augment_target=None
)
    
# training loop
for epoch in range(1, model_config["epochs"] + 1):
    print(f"Epoch: {epoch}/{model_config['epochs']}")

    train_f1, train_loss = train(
        device=device,
        dataloader=training_dataloader,
        optimizer=optimizer,
        model=model,
        loss=loss,
        mixer=mixer
    )

    valid_f1, valid_loss = evaluate(
        device=device,
        dataloader=validation_dataloader,
        model=model,
        loss=loss
    )
    
    report = test(
        model=model,
        dataloader=test_dataloader,
        device=device
    )

    test_f1 = report["macro avg"]["f1-score"]

    print(f"\nEpoch: {epoch}")
    print(f"Train F1-Score: {train_f1:1.6f}")
    print(f"Train Loss: {train_loss:1.6f}")
    print(f"Validation F1-Score: {valid_f1:1.6f}")
    print(f"Validation Loss: {valid_loss:1.6f}")
    print(f"Test F1-Score: {test_f1:1.6f}\n")

    # updating learning rate
    if not scheduler is None:
        scheduler.step()

Epoch: 1/100

Epoch: 1
Train F1-Score: 0.313523
Train Loss: 1.811929
Validation F1-Score: 0.300420
Validation Loss: 1.825701
Test F1-Score: 0.366102

Epoch: 2/100

Epoch: 2
Train F1-Score: 0.339942
Train Loss: 1.393618
Validation F1-Score: 0.327557
Validation Loss: 1.467577
Test F1-Score: 0.371932

Epoch: 3/100

Epoch: 3
Train F1-Score: 0.337208
Train Loss: 1.337283
Validation F1-Score: 0.291291
Validation Loss: 1.607207
Test F1-Score: 0.337789

Epoch: 4/100

Epoch: 4
Train F1-Score: 0.321041
Train Loss: 1.272540
Validation F1-Score: 0.354012
Validation Loss: 1.453305
Test F1-Score: 0.326450

Epoch: 5/100

Epoch: 5
Train F1-Score: 0.357263
Train Loss: 1.075650
Validation F1-Score: 0.316564
Validation Loss: 1.259725
Test F1-Score: 0.386010

Epoch: 6/100

Epoch: 6
Train F1-Score: 0.332198
Train Loss: 1.222494
Validation F1-Score: 0.468241
Validation Loss: 1.119771
Test F1-Score: 0.337640

Epoch: 7/100

Epoch: 7
Train F1-Score: 0.334489
Train Loss: 1.149200
Validation F1-Score: 0.289193
V

/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu


Epoch: 42
Train F1-Score: 0.394556
Train Loss: 0.723894
Validation F1-Score: 0.294643
Validation Loss: 0.986761
Test F1-Score: 0.297232

Epoch: 43/100

Epoch: 43
Train F1-Score: 0.368484
Train Loss: 0.668103
Validation F1-Score: 0.376409
Validation Loss: 0.970149
Test F1-Score: 0.396280

Epoch: 44/100

Epoch: 44
Train F1-Score: 0.354194
Train Loss: 0.732666
Validation F1-Score: 0.292609
Validation Loss: 1.062851
Test F1-Score: 0.354321

Epoch: 45/100

Epoch: 45
Train F1-Score: 0.410496
Train Loss: 0.677237
Validation F1-Score: 0.388032
Validation Loss: 0.945430
Test F1-Score: 0.322414

Epoch: 46/100

Epoch: 46
Train F1-Score: 0.375210
Train Loss: 0.718546
Validation F1-Score: 0.333001
Validation Loss: 1.078442
Test F1-Score: 0.340050

Epoch: 47/100

Epoch: 47
Train F1-Score: 0.379269
Train Loss: 0.657019
Validation F1-Score: 0.380535
Validation Loss: 1.000965
Test F1-Score: 0.322958

Epoch: 48/100

Epoch: 48
Train F1-Score: 0.385117
Train Loss: 0.680528
Validation F1-Score: 0.423909
V

/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu


Epoch: 56
Train F1-Score: 0.377545
Train Loss: 0.679462
Validation F1-Score: 0.294643
Validation Loss: 0.974003
Test F1-Score: 0.296029

Epoch: 57/100

Epoch: 57
Train F1-Score: 0.343606
Train Loss: 0.669379
Validation F1-Score: 0.445000
Validation Loss: 0.968504
Test F1-Score: 0.351615

Epoch: 58/100

Epoch: 58
Train F1-Score: 0.371233
Train Loss: 0.665736
Validation F1-Score: 0.362293
Validation Loss: 0.914923
Test F1-Score: 0.296564

Epoch: 59/100

Epoch: 59
Train F1-Score: 0.396108
Train Loss: 0.632827
Validation F1-Score: 0.295626
Validation Loss: 1.028401
Test F1-Score: 0.329490

Epoch: 60/100

Epoch: 60
Train F1-Score: 0.372337
Train Loss: 0.665757
Validation F1-Score: 0.389235
Validation Loss: 0.858317
Test F1-Score: 0.294545

Epoch: 61/100


...

/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu


Epoch: 63
Train F1-Score: 0.397671
Train Loss: 0.616839
Validation F1-Score: 0.291291
Validation Loss: 1.096288
Test F1-Score: 0.315530

Epoch: 64/100

Epoch: 64
Train F1-Score: 0.388541
Train Loss: 0.612763
Validation F1-Score: 0.351549
Validation Loss: 0.988618
Test F1-Score: 0.360195

Epoch: 65/100

Epoch: 65
Train F1-Score: 0.383311
Train Loss: 0.638318
Validation F1-Score: 0.388485
Validation Loss: 1.131001
Test F1-Score: 0.397373

Epoch: 66/100

Epoch: 66
Train F1-Score: 0.381014
Train Loss: 0.635248
Validation F1-Score: 0.360462
Validation Loss: 0.889018
Test F1-Score: 0.333973

Epoch: 67/100

Epoch: 67
Train F1-Score: 0.396950
Train Loss: 0.626865
Validation F1-Score: 0.352844
Validation Loss: 0.921117
Test F1-Score: 0.369532

Epoch: 68/100


/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu


Epoch: 68
Train F1-Score: 0.391065
Train Loss: 0.603644
Validation F1-Score: 0.360961
Validation Loss: 0.996103
Test F1-Score: 0.320585

Epoch: 69/100

Epoch: 69
Train F1-Score: 0.380248
Train Loss: 0.611115
Validation F1-Score: 0.523829
Validation Loss: 0.825975
Test F1-Score: 0.360261

Epoch: 70/100

Epoch: 70
Train F1-Score: 0.390864
Train Loss: 0.636605
Validation F1-Score: 0.330303
Validation Loss: 0.945265
Test F1-Score: 0.361286

Epoch: 71/100

Epoch: 71
Train F1-Score: 0.382622
Train Loss: 0.601570
Validation F1-Score: 0.292609
Validation Loss: 0.937060
Test F1-Score: 0.339503

Epoch: 72/100

Epoch: 72
Train F1-Score: 0.377077
Train Loss: 0.675140
Validation F1-Score: 0.481340
Validation Loss: 0.757064
Test F1-Score: 0.369532

Epoch: 73/100

Epoch: 73
Train F1-Score: 0.376390
Train Loss: 0.635483
Validation F1-Score: 0.340370
Validation Loss: 0.986767
Test F1-Score: 0.357197

Epoch: 74/100


KeyboardInterrupt: 